In [1]:
import pandas as pd
import numpy as np

import time
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_kt = pd.read_csv('./data/dp_frc_bldg_ftc_pred_slng_m_sum_202205.csv',sep='|')

In [3]:
df_open = pd.read_excel('./data/4_6월 오픈점리스트.xlsx')

### 도로명 주소 추출

In [146]:
# 도로명 주소 추출
df_open['도로명주소'] = 0

for i in df_open.index:
    df_open['도로명주소'][i] = df_open['주소전체'][i].split('(')[-1][:-1]

### 시군구 포맷 변경

In [147]:
# 시군구 풀네임 추출
sido = df_kt.drop_duplicates('sido_nm')[['sido_nm']]
sido = sido.reset_index()
sido = sido.drop(columns=['index'])

In [148]:
# 세종시 주소가 충남으로 되어있음 -> 주소 수정 후 sido 담기
df_open['sido'] = 0
idx_sj = df_open[df_open['주소전체'].str.contains('세종시')].index
df_open['sido'][idx_sj] = df_open['주소전체'][idx_sj].str.split(' ').str[1]

In [149]:
# 나머지 지역 sido 담기
idx = df_open[~df_open['주소전체'].str.contains('세종시')].index
df_open['sido'][idx] = df_open['주소전체'][idx].str.split(' ').str[0]

In [150]:
# 시도 이름 포맷 변경 (앞자리만 딴 지역)
for sido_idx in sido['sido_nm'][:10].index:
    idx = df_open[df_open['sido'].str.contains(sido['sido_nm'][sido_idx][:2], na=False)].index
    df_open['sido'][idx] = sido['sido_nm'][sido_idx]
    
# 시도 이름 포맷 변경 (세종)
for sido_idx in sido['sido_nm'][-1:].index:
    idx = df_open[df_open['sido'].str.contains(sido['sido_nm'][sido_idx][:2], na=False)].index
    df_open['sido'][idx] = sido['sido_nm'][sido_idx]
    
# 시도 이름 포맷 변경 (나머지 지역)
for sido_idx in sido['sido_nm'][10:16].index:
    idx = df_open[df_open['sido'].str.contains(sido['sido_nm'][sido_idx][0] + sido['sido_nm'][sido_idx][2], na=False)].index
    df_open['sido'][idx] = sido['sido_nm'][sido_idx]

###### 테스트 -> 오류남

In [151]:
# 시도 이름 포맷 변경 (한 번에 변경)
for sido_idx in sido['sido_nm'].index:
    if sido_idx == sido['sido_nm'][:10].index or sido_idx == sido['sido_nm'][-1:].index:
        idx = df_open[df_open['sido'].str.contains(sido['sido_nm'][sido_idx][:2], na=False)].index
        df_open['sido'][idx] = sido['sido_nm'][sido_idx]
    else :
        idx = df_open[df_open['sido'].str.contains(sido['sido_nm'][sido_idx][0] + sido['sido_nm'][sido_idx][2], na=False)].index
        df_open['sido'][idx] = sido['sido_nm'][sido_idx]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
# 시도 이름 포맷 변경
def sido_f(a,*b):
    for sido_idx in sido['sido_nm'][a:b].index:
        idx = df_open[df_open['sido'].str.contains(sido['sido_nm'][sido_idx][:2], na=False)].index
        df_open['sido'][idx] = sido['sido_nm'][sido_idx]
    
# sido(0,10)

### 전체 주소 병합

In [152]:
# 전체 주소 병합
idx    = df_open[~df_open['주소전체'].str.contains('제주시')].index
idx_jj = df_open[df_open['주소전체'].str.contains('제주시')].index
df_open['도로명주소'][idx]    = df_open['sido'][idx] + ' ' + df_open['주소전체'][idx].str.split(' ').str[1] + ' ' + df_open['도로명주소'][idx]
df_open['도로명주소'][idx_jj] = df_open['sido'][idx_jj] + ' ' + df_open['주소전체'][idx_jj].str.split(' ').str[1]+ ' ' + df_open['주소전체'][idx_jj].str.split(' ').str[2] + ' ' + df_open['도로명주소'][idx_jj]

In [471]:
df_open

,NO.,점포명,주소전체,도로명주소,sido,bd_mgt_sn
0,1,오산원동대림점,경기 오산시 원동 900(오산로132번길 8),경기도 오산시 오산로132번길 8,경기도,148535 4137010300109000000000003 Name: bd_m...
1,2,병점스테이점,"경기 화성시 병점동 374-5, 101호(떡전골로 72-6)",경기도 화성시 떡전골로 72-6,경기도,4420025325115710000000001
2,3,정남만년로점,경기 화성시 정남면 괘랑리 305-11(괘랑리 305-11),경기도 화성시 괘랑리 305-11,경기도,"Series([], Name: bd_mgt_sn, dtype: object)"
3,4,화성남양양우점,"경기 화성 남양읍 남양리 2294-9 103,104호(시청로 66)",경기도 화성 시청로 66,경기도,762891 4715010800103940004003428 887818 ...
4,5,안양미도점,경기 안양시 만안구 안양동 371-6(안양로 29),경기도 안양시 안양로 29,경기도,21155 4117110100106760215007997 221988 ...
...,...,...,...,...,...,...
424,425,부안하이스트점,"전북 부안군 부안읍 서외리 446-3, 상가동 114호, 115호(번영로 63-10)",전라북도 부안군 번영로 63-10,전라북도,"Series([], Name: bd_mgt_sn, dtype: object)"
425,426,제주외도부영점,제주 제주시 외도일동 550-10(우정로11길 12),제주특별자치도 제주시 외도일동 우정로11길 12,제주특별자치도,603022 5011012300105500010000001 Name: bd_m...
426,427,서귀포신천리점,제주 서귀포시 성산읍 신천리 220-10(신천로49번길 32),제주특별자치도 서귀포시 신천로49번길 32,제주특별자치도,"Series([], Name: bd_mgt_sn, dtype: object)"
427,428,서귀포신산리점,제주 서귀포시 성산읍 신산리 607-2(일주동로 5065),제주특별자치도 서귀포시 일주동로 5065,제주특별자치도,945236 5013025927106070002000003 Name: bd_m...


### KT 병합

In [153]:
# 건물 기준 중복 제거
df_kt = df_kt.drop_duplicates('bd_mgt_sn')
df_kt = df_kt.reset_index(drop=True)

In [466]:
# 도로명 주소 포맷 맞추기
df_kt[['buld_mnnm','buld_slno']] = df_kt[['buld_mnnm','buld_slno']].astype(str)
df_kt['도로명주소'] = df_kt['sido_nm'] + ' ' + df_kt['sig_nm'] + ' ' + df_kt['rn'] + ' ' + df_kt['buld_mnnm'] + '-' + df_kt['buld_slno']

In [362]:
# 주소 매칭
df_open['bd_mgt_sn'] = 0
idx_co = df_open[df_open['도로명주소'].isin(df_kt['도로명주소'])].index
idx_in = df_open[~df_open['도로명주소'].isin(df_kt['도로명주소'])].index # df_kt에 없는 주소 인덱스 담음

In [376]:
# 주소 일치하는 경우
idx = df_kt[df_kt['도로명주소'].isin(df_open['도로명주소'][idx_co])].index
df_open['bd_mgt_sn'][idx_co] = df_kt['bd_mgt_sn'][idx].astype(str)

In [491]:
a = []

In [493]:
# 주소가 일치하지 않는 경우
for i in idx_in:
    name_ = df_open.loc[i,'도로명주소'].split(' ')[-2] + ' ' + df_open.loc[i,'도로명주소'].split(' ')[-1] # 도로명 주소(00번길 0-0)만 추출
    try:
        idx = df_kt[df_kt['도로명주소'].str.contains(name_, na=False)].index # df_kt에 일치하는 주소가 있는 지 확인
        df_open['bd_mgt_sn'][i] = list(df_kt['bd_mgt_sn'])[idx[0]] # 해당 인덱스에 건물번호값 대입
    except:
        a.append(i)
        pass

In [472]:
df_kt['bd_mgt_sn'][2]

'1156013200102040006011908'

In [487]:
list(df_kt['bd_mgt_sn'])[148535]

'4137010300109000000000003'

In [481]:
idx = df_kt[df_kt['도로명주소'].str.contains('경기도 오산시 오산로132번길 8', na=False)].index

In [500]:
df_open[df_open['bd_mgt_sn'].str.contains('Series', na=False)]

,NO.,점포명,주소전체,도로명주소,sido,bd_mgt_sn


In [513]:
df_open[df_open['bd_mgt_sn'].astype(str).str.contains('Series')]

,NO.,점포명,주소전체,도로명주소,sido,bd_mgt_sn
2,3,정남만년로점,경기 화성시 정남면 괘랑리 305-11(괘랑리 305-11),경기도 화성시 괘랑리 305-11,경기도,"Series([], Name: bd_mgt_sn, dtype: object)"
6,7,봉천샛별점,서울 관악구 봉천동 968-3(봉천로 295-1),서울특별시 관악구 봉천로 295-1,서울특별시,"Series([], Name: bd_mgt_sn, dtype: object)"
8,9,상도아람점,서울 동작구 상도동 301-40(국사봉8길77),서울특별시 동작구 국사봉8길77,서울특별시,"Series([], Name: bd_mgt_sn, dtype: object)"
9,10,경기드림타워점,경기 수원시 이의동 934-11 경기드림타워1층(광교산로 154-42),경기도 수원시 광교산로 154-42,경기도,"Series([], Name: bd_mgt_sn, dtype: object)"
10,11,산본한라2차점,경기 군포시 산본동 1156-25(번영로330),경기도 군포시 번영로330,경기도,"Series([], Name: bd_mgt_sn, dtype: object)"
...,...,...,...,...,...,...
411,412,창녕남지유채점,경남 창녕군 남지읍 남지리 979-1(남지중앙1긹 56),경상남도 창녕군 남지중앙1긹 56,경상남도,"Series([], Name: bd_mgt_sn, dtype: object)"
412,413,진도조도점,전남 진도군 조도면 창유리 2876(어류포길1-4),전라남도 진도군 어류포길1-4,전라남도,"Series([], Name: bd_mgt_sn, dtype: object)"
421,422,전주빌리언점,전북 전주시 완산구 서서학동 서서학동117-2(공수내로19),전라북도 전주시 공수내로19,전라북도,"Series([], Name: bd_mgt_sn, dtype: object)"
424,425,부안하이스트점,"전북 부안군 부안읍 서외리 446-3, 상가동 114호, 115호(번영로 63-10)",전라북도 부안군 번영로 63-10,전라북도,"Series([], Name: bd_mgt_sn, dtype: object)"


In [514]:
df_open.to_excel('./4_6월 오픈점리스트_건물번호(120개 남음).xlsx', index=False)

In [515]:
DF = pd.read_excel('./4_6월 오픈점리스트_건물번호(120개 남음).xlsx')

###### test

In [516]:
# 부번이 -0으로 들어간 경우
df_sub = df_open[~df_open['도로명주소'].isin(df_kt['도로명주소'])]
df_sub = df_sub.reset_index(drop=True)
idx_0  = []
nm_list= []

In [338]:
for i in df_sub.index:
    idx_0.append(df_kt[[df_kt['도로명주소'].str.contains(df_sub['도로명주소'][i], na=False)]].index)
    nm_list.append(i)

In [342]:
df_sub['bd_mgt_sn'] = 0
df_sub['bd_mgt_sn'][i] = df_kt.loc[idx_0,'bd_mgt_sn']

TypeError: unhashable type: 'Int64Index'

In [517]:
# 주소 수정 후 다시 돌림
df_open['도로명주소'][i] = df_open['도로명주소'][idx_sub].str.split(')').str[0]

###### test

### 세부주소 포맷
(([가-힣]+(d|)+(읍|면|동|가|리)) # 읍면동 찾기
( |)((d(~|-)d|d)(가|리|)|)) # 리
([ ](산(d(~|-)d|d))|)|(([가-R]|(d(~|-)d)|d)+(로|길)) #로길
+[ ,]+
(((d)|(d번지)|(d(~|-)d)|(d(~|-)d번지))|(((지하)d)|((지하)d번지)|((지하)d(~|-)d)|((지하)d(~|-)d번지)))

In [ ]:
import re

In [10]:
df_open

,NO.,점포명,주소전체
0,1,오산원동대림점,경기 오산시 원동 900(오산로132번길 8)
1,2,병점스테이점,"경기 화성시 병점동 374-5, 101호(떡전골로 72-6)"
2,3,정남만년로점,경기 화성시 정남면 괘랑리 305-11(괘랑리 305-11)
3,4,화성남양양우점,"경기 화성 남양읍 남양리 2294-9 103,104호(시청로 66)"
4,5,안양미도점,경기 안양시 만안구 안양동 371-6(안양로 29)
...,...,...,...
424,425,부안하이스트점,"전북 부안군 부안읍 서외리 446-3, 상가동 114호, 115호(번영로 63-10)"
425,426,제주외도부영점,제주 제주시 외도일동 550-10(우정로11길 12)
426,427,서귀포신천리점,제주 서귀포시 성산읍 신천리 220-10(신천로49번길 32)
427,428,서귀포신산리점,제주 서귀포시 성산읍 신산리 607-2(일주동로 5065)


In [551]:
# 시도 검색
regex = r'(([가-힣]+(시|도))( |)[가-힣]+(시|군|구))'
test  = '제주특별자치도 서귀포시 일주동로 5065'
x = re.search(regex, test)
x.group()

'제주특별자치도 서귀포시'

In [558]:
# 시도 검색 ver2
regex = r'(([가-힣]+(시|도)|[서울]|[인천]|[대구]|[광주]|[부산]|[울산])( |)[가-힣]+(시|군|구))|'
test  = '제주특별자치도 서귀포시 일주동로 5065'
x = re.search(regex, test)
x.group()

'제주특별자치도 서귀포시'

In [560]:
# 읍면동 검색
regex = r'([가-힣]+(d|d(,|.)d|)+(읍|면|동|가|리))(^구|)'
test  = '제주특별자치도 서귀포시 일주동로 5065'
x = re.search(regex, test)
x.group()

'일주동'

In [566]:
# 읍면동 검색
regex = r'((d(~|-)d|d)(가|리|)|)|(([가-힣]|(d(~|-)d)|d)+(로|길))'
test  = '제주특별자치도 서귀포시 일주리 일주동로 5065'
x = re.search(regex, test)
x.group()

''

In [528]:
# regex = r'(\w+[시,도]\s*)?(\w+[구,시,군]\s*)?(\w+[구,시,로]\s*)?(\w+[면,읍]\s*)?(\w+\d*\w*[동,리,로,길]\s*)?(\w*\d+-?\d*)?' # 법정동 포맷
# regex = r'(\w+[시,도]\s*)?(\w+[구,시,군]\s*)?(\w+[구,시,로]\s*)?(\w+\d*\w*[리,로,길]\s*)?(\w+\d*\w*[길]\s*)?(\w*\d+-?\d*)?' # 도로명 포맷
regex = r'(\w+\d*\w*[길]\s*)?(\w*\d+-?\d*)?' # 도로명(00길 0-0) 포맷
test  = '제주특별자치도 서귀포시 일주동로 5065'
x = re.search(regex, test)
x.group()

''

In [99]:
# regex = r'(\w+\d*\w*[로]\s*)?(\w*\d+-?\d*)?'

In [104]:
all_ = []

In [105]:
all_values = df_open[~df_open['도로명주소'].isin(df_kt['도로명주소'])]['도로명주소']
for s in all_values:
    x = re.search(regex, s)
    all_.append(x.group())

In [107]:
df_open[~df_open['도로명주소'].isin(df_kt['도로명주소'])]['도로명주소']

0              경기도 오산시 오산로132번길 8
2              경기도 화성시 괘랑리 305-11
3                   경기도 화성 시청로 66
4                  경기도 안양시 안양로 29
5                경기도 수원시 서수원로 607
                  ...            
424            전라북도 부안군 번영로 63-10
425    제주특별자치도 제주시 외도일동 우정로11길 12
426       제주특별자치도 서귀포시 신천로49번길 32
427        제주특별자치도 서귀포시 일주동로 5065
428     제주특별자치도 제주시 이도일동 중앙로19길 2
Name: 도로명주소, Length: 410, dtype: object

In [106]:
all_

['경기도 오산시 오산로132번길 8',
 '경기도 화성시 괘랑리 305-11',
 '경기도 ',
 '경기도 안양시 안양로 29',
 '경기도 수원시 서수원로 607',
 '서울특별시 관악구 봉천로 295-1',
 '서울특별시 관악구 ',
 '서울특별시 동작구 국사봉8길77',
 '경기도 수원시 광교산로 154-42',
 '경기도 군포시 번영로330',
 '경기도 군포시 용호1로5번길11',
 '경기도 군포시 당동로 32번길 25',
 '인천광역시 부평구 새갈로 10',
 '인천광역시 부평구 주부토로 236',
 '인천광역시 부평구 주부토로 236',
 '인천광역시 중구 운중로175',
 '인천광역시 남동구 문화서로17번길6',
 '인천광역시 중구 하늘달빛로 95',
 '인천광역시 부평구 주부토로 69',
 '인천광역시 계양구 경명대로 ',
 '인천광역시 계양구 계양대로 ',
 '인천광역시 서구 ',
 '인천광역시 서구 대곡로123번길 4',
 '인천광역시 미추홀구 경원대로864번길12',
 '인천광역시 동구 연송로 4',
 '경기도 남양주시 남가로 1817',
 '경기도 포천시 금강로 3137',
 '경기도 남양주시 진건오남로884번길23',
 '경기도 의정부시 세석로20',
 '경기도 남양주시 수레로 587',
 '경기도 가평군 비룡로 1972',
 '서울특별시 도봉구 시루봉로 205',
 '서울특별시 성북구 장위로 38',
 '서울특별시 강북구 노해로 56',
 '서울특별시 서대문구 응암로 126',
 '서울특별시 중구 무교로13',
 '서울특별시 성북구 서경로28',
 '경기도 동두천시 동두천로175',
 '경기도 파주시 자유로 2210',
 '경기도 양주시 그루고개로210',
 '경기도 양주시 부흥로 2162',
 '경기도 고양시 통일로 1282',
 '경기도 양주시 평화로 1261번길 82',
 '경기도 고양시 호수로 856번길 56-1',
 '경기도 김포시 봉화로182번길 33',
 '경기도 광주시 봉골길 86-1',
 '서울특별시 송파구 백제고분

In [21]:
df_kt[df_kt['도로명주소'].str.contains(test, na=False)]

,base_ym,bd_mgt_sn,sido_cd,sido_nm,sig_cd,sig_nm,emd_cd,emd_nm,rn_cd,rn,buld_mnnm,buld_slno,ftc_cate2_cd,pred_slng_amt,도로명주소


In [ ]:
# 주소 쪼개서 검색
idx = df_kt[df_kt['도로명주소'].str.contains('서울특별시 영등포구', na=False) & df_kt['도로명주소'].str.contains('도신로48길', na=False) & df_kt['도로명주소'].str.contains('11-2', na=False)].index

인덱스 추출 -> 값 추출

In [ ]:
# 부번이 -0으로 들어간 경우 -> 값으로 들어가지 않고 
idx_sub = []
for i in idx_in:
    try:
        idx_0 = df_kt[df_kt['도로명주소'].str.contains(df_open['도로명주소'][i], na=False)].index
        df_open['bd_mgt_sn'][i] = df_kt.loc[idx_0,'bd_mgt_sn'].values[0]
    except:
        print(i)
        idx_sub.append(i)
        pass

# 주소 수정 후 다시 돌림
df_open['도로명주소'][idx_sub] = df_open['도로명주소'][idx_sub].str.split(')').str[0]
for i in idx_sub:
    try:
        idx_0 = df_kt[df_kt['도로명주소'].str.contains(df_open['도로명주소'][i], na=False)].index
        df_open['bd_mgt_sn'][i] = df_kt.loc[idx_0,'bd_mgt_sn'].values[0]
    except:
        print(i)
        idx_sub.append(i)
        pass

### 법정동 변환/수집 (크레마오에서 크롤링)

In [15]:
import time
from tqdm import tqdm

import requests
import urllib
from urllib.request import urlopen
from html_table_parser import parser_functions

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.select import Select

DF명, XPath 수정 후 사용

In [184]:
# 법정동명, 법정동 코드 가져오기
def get_text():
    bcode.append(driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/ul/li').get_attribute('data-bcode'))
    sido    = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/ul/li').get_attribute('data-sido')    # 시도
    sigungu = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/ul/li').get_attribute('data-sigungu') # 군구
    bname   = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/ul/li').get_attribute('data-bname2')  # 법정동
    lgdng_nm_list.append(sido + ' ' + sigungu + ' ' + bname)
    lgdng_full_nm_list.append(driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/ul/li/dl/dd[1]/span/button/span[1]').text)


In [185]:
# 주소 검색
def search():
    miss_index = []
    for i in tqdm(df_open.index):
        searchByName = driver.find_element_by_xpath('//*[@id="region_name"]')                        # 검색창 선택
        searchByName.clear()                                                                         # 기존 입력 삭제
        
        time.sleep(1)
        
        searchByName.send_keys(df_open['주소전체'][i])                                               # 검색어 입력
        click = driver.find_element_by_xpath('//*[@id="searchForm"]/fieldset/div/button[2]').click() # 검색 버튼 클릭
        
        time.sleep(1)
        
        try:
            get_text()                                                                                # 주소 가져오기
            corp_nm.append(df_open['점포명'][i])
        except:
            miss_index.append(i)
            pass


In [186]:
# 기본 틀
driver = webdriver.Chrome('./driver/chromedriver.exe')
driver.get('https://postcode.map.daum.net/search?origin=https%3A%2F%2Fdev.cremao.co.kr&indaum=off&banner=on&ubl=on&mode=transmit&vt=layer&amr=on&amj=on&ani=off&sd=off&fi=on&fc=on&plrg=&plrgt=1.5&hmb=off&heb=off&asea=off&smh=off&zo=on&us=on&msi=10&ahs=off&whas=500&sm=on&a51=off&theme=BODY.bg_FFFFFF-SEARCH.bg_FFFFFF.placeHolder_bbbbbb.bottomLine_858585-LOGO.text_888888.logoTextDividingLine_cccccc&bit=0&sit=0&sgit=0&sbit=0&pit=0&mit=0&lcit=0&zn=Y&fullpath=%2Fmain')

# 주소 수집
bcode = []              # 법정동 코드
corp_nm = []            # 매장명
lgdng_nm_list = []      # 법정동 명
lgdng_full_nm_list = [] # 법정동 전체 주소
search()

driver.close()

# 저장
df = pd.DataFrame({'corp_nm':corp_nm, 'lgdng_nm_list':lgdng_nm_list,'lgdng_full_nm_list':lgdng_full_nm_list,'bcode':bcode})
df.to_excel('./법정동주소.xlsx', index=False)

100%|████████████████████████████████████████████████████████████████████████████████| 429/429 [15:56<00:00,  2.23s/it]


###### 제대로 안 담겼을 때

In [151]:
df = pd.DataFrame({'lgdng_nm_list':lgdng_nm_list,'lgdng_full_nm_list':lgdng_full_nm_list})

In [153]:
corp_nm = pd.DataFrame({'bcode':bcode})

In [132]:
corp_nm = pd.DataFrame({'corp_nm':corp_nm})

In [101]:
lgdng_nm.to_excel('./test.xlsx', index= False)

###### test

In [134]:
driver = webdriver.Chrome('./driver/chromedriver.exe')
driver.get('https://postcode.map.daum.net/search?origin=https%3A%2F%2Fdev.cremao.co.kr&indaum=off&banner=on&ubl=on&mode=transmit&vt=layer&amr=on&amj=on&ani=off&sd=off&fi=on&fc=on&plrg=&plrgt=1.5&hmb=off&heb=off&asea=off&smh=off&zo=on&us=on&msi=10&ahs=off&whas=500&sm=on&a51=off&theme=BODY.bg_FFFFFF-SEARCH.bg_FFFFFF.placeHolder_bbbbbb.bottomLine_858585-LOGO.text_888888.logoTextDividingLine_cccccc&bit=0&sit=0&sgit=0&sbit=0&pit=0&mit=0&lcit=0&zn=Y&fullpath=%2Fmain')